### Imports

In [1]:
import numpy as np
import datetime
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (10.0, 7.0)
from matplotlib.colors import LogNorm

### Extracting data from .mpa files

In [ ]:
def get_spectrum(filename, start_line):    
    '''
    Get the spectrum of ADC1 in .mpa file. 
    Start line: contains the line [DATA0, 1024] or something similar in the .mpa file.
    This may be determined using the function 'get_line_start'.
    End line determined automatically from this line.
    Returns list containing spectrum. In case of 2d spectrum some reshaping might be in order. 
    I suggest: np.reshape(list, (256,256)) :)
    OR: np.reshape(list, (1024,4096))
    '''
    spec = []
    with open(filename) as f:
        # Loop over lines and keep track of line number
        for i, line in enumerate(f):
            if i == (start_line - 1):
                try:                    
                    # Unreadable line splits off part before ',' and part after ']'
                    n_samples = (int(line.split(sep=',')[1].split(sep=']')[0]))
                except:
                    raise RuntimeError("Error: the line you indicated contains: %s" % line)
            if i >= start_line:
                if i < start_line + n_samples:
                    spec.append(int(line))
    # Do some checking
    print('Spectrum containing %d samples read, total %d counts' % (len(spec), sum(spec)))
    if(sum(spec)==0):
        print("Warning: empty spectrum")
    return spec

In [ ]:
def find_line_start(filename, string):
    '''
    Find the line in the text file that starts with the string supplied. (first occurrance)
    Returns -1 if string not found.
    The output of this may be used as imput to get_spectrum
    '''
    
    with open(filename) as f:
        # Loop over lines and keep track of line number
        for i, line in enumerate(f):
            if len(line) < len(string):
                continue
            if line[:len(string)] == string:
                return i + 1
    return -1

In [ ]:
def find_full_line(filename, string):
    '''
    Find the line in the text file that starts with the string supplied. (first occurrance)
    Returns the full line
    '''
    
    with open(filename) as f:
        # Loop over lines and keep track of line number
        for i, line in enumerate(f):
            if len(line) < len(string):
                continue
            if line[:len(string)] == string:
                return line
    return ''

In [ ]:
def get_mpa_entry(filename, string, endmarker='\n', startmarker='='):
    '''
    Get the entry of a certain property in the .mpa file
    String: start of the line to be extracted
    '''
    line = find_full_line(filename, string) # Get the full line
    line = line.split(sep=startmarker)[1] # Split off everything before '='
    line = line.split(sep=endmarker)[0] # Split off everything before endmarker
    return line

In [ ]:
def get_mpa_data(fn):
    '''
    Get the following data from the .mpa file:
      - 
      - 
    WARNING: if a line occurs multiple times in the file, we will take the FIRST one!
    '''
    # Initialize dictionary that is to hold all data
    d = {}
    
    
    d['runtime'] = float(get_mpa_entry(fn, 'scrtime'))
    d['gm_counts'] = int(get_mpa_entry(fn, 'sc#05', ';'))
    d['lc_counts'] = int(get_mpa_entry(fn, 'sc#07', ';'))
    d['livetime'] = float(get_mpa_entry(fn, 'livetime'))
    d['start_time_seconds'] = float(get_mpa_entry(fn, 'scrtstart', endmarker=';'))
    d['start_datetime'] = convert_mpa_time(
                            ' ' + get_mpa_entry(fn, 'scrtstart', startmarker=';')) # ' ' for backward compatibility
    d['end_datetime'] = d['start_datetime'] + datetime.timedelta(seconds = d['runtime'])
    
    return d

### Reading data from .msa files

In [ ]:
def read_msa_data(count_file, verbose = True):
    counts = []
    with open(count_file) as f:
        i = 0
        for line in f:
            # Don't read the header
            if i==0:
                i = i+1
                continue
            # Split on spaces
            split_line = np.array(line.split(sep=' '))
            # Remove all the extra entries caused by spaces
            split_line = [x for x in split_line if x != '']

            dt = np.dtype([
                    ('date',np.str_,20),
                    ('time',np.str_,20),
                    ('interval', np.float),
                    ('gm_counts', np.int),
                    ('lc_counts', np.int),
                    ('end', datetime.datetime),
                    ('start', datetime.datetime),
                  ])
            d = np.zeros(1,dtype=dt)
            d['date']     = str(split_line[1])
            d['time']     = str(split_line[2])
            d['interval'] = 0.01*float(split_line[3])
            d['end']      = convert_count_time(d['date'][0],d['time'][0])
            d['start']    = d['end'][0] - datetime.timedelta(seconds = d['interval'][0])
            d['gm_counts']= int(split_line[7])
            d['lc_counts']= int(split_line[9])
            counts.append(d)

            i = i+1
    counts = np.concatenate(counts)

    if verbose: print("Read %d datasets" % (len(counts)))
    return counts

### Converting fucking timestamps

In [2]:
def convert_mpa_time(s):
    '''
    Take a string in the format of mpa file and convert it to something sensible.
    Example string: '; 05/30/2016 16:46:00\n'
    '''
    month = int(s[2:4])
    day = int(s[5:7])
    year = int(s[8:12])
    hour = int(s[13:15])
    minute = int(s[16:18])
    second = int(s[19:21])
    return datetime.datetime(year,month,day,hour,minute,second)

In [ ]:
def convert_sc_time(s):
    '''
    Take a string in the format of SC program and convert it to something sensible.
    Example string: "b'25.05.2016 13:30:56"
    '''
    # Cut off b' part
    s = s[2:]
    day = int(s[0:2])
    month = int(s[3:5])
    year = int(s[6:10])
    hour = int(s[11:13])
    minute = int(s[14:16])
    second = int(s[17:19])
    return datetime.datetime(year,month,day,hour,minute,second)

In [ ]:
def convert_count_time(date,time):
    '''
    Take string in format of counter and convert it.
    '25-MAY-2016', '15:44:54'
    '''
    day = int(date[0:2])
    month_str = (date[3:6])
    if month_str == 'MAY':
        month = 5
    else:
        raise ValueError("What? How dare you operate outside of May?!")
    year = int(date[-4:])
    hour = int(time[0:2])
    minute = int(time[3:5])
    second = int(time[6:8])
    return datetime.datetime(year,month,day,hour,minute,second)

### 2D histogram manipulation

In [ ]:
def rebin_x(arr2d):
    '''
    Rebin 2d histogram by a factor of 2 in the x-direction
    '''
    return np.array(
        [arr2d[i] + arr2d[i +1] for i in np.arange(0,len(arr2d),2)]
    )

def rebin_y(arr2d):
    '''
    Rebin 2d histogram by a factor of 2 in the y-direction
    '''
    return np.array(
    [ [x[i] + x[i + 1] for i in np.arange(0,len(x),2)]  for x in arr2d       
        ])

In [ ]:
def cut_hist2d(arr2d, f):
    '''
    Set bin content to 0 whenever f(x,y) yields False
    '''
    return np.array([
            
            [arr2d[y][x] if f(x,y) else 0 for x in range(len(arr2d[y]))]
            for y in range(len(arr2d))
        ])

In [ ]:
def get_percentile_pos(dist, percent):
    '''
    Get the position in the list containing histogram of the percentile
    No, np.percentile does NOT work, since we're dealing with the histogram rather than the values themselves!
    If you care about +-1 accuracy, better check what makes sense because I didn't.
    This takes lower bound.
    '''
    tot = sum(dist)
    count = 0
    for i, x in enumerate(dist):
        if count >= tot*percent/100.:
            return i
        count += x
    return -1

### Slow control data

In [ ]:
def read_sc_data(filename):
    # Load SC data
    dt = np.dtype([('date-time',np.str_,21),
                   ('v', np.float64),
                   ('v_set', np.float64),
                   ('i', np.float64),
                   ('i_set', np.float64),
                   ('temp_basic', np.float64),
                   ('temp_coset', np.float64),
                   ('temp', np.float64),
                  ])
    sc_data = np.loadtxt(filename, dtype=dt, delimiter = ',', skiprows=1, usecols=(0,1,2,4,5,8,9,10))
    
    # Convert the STRING date-time to a datetime.datetime timestamp
    t = np.array([convert_sc_time(datetime) for datetime in sc_data['date-time'] ])
    
    return t, sc_data

In [ ]:
def get_correction_factor(t, sc_data, start_datetime, end_datetime, verbose=True, plot=False):
    '''
    Get the correction factor based on voltage and current readings.
    filename: SC file name
    '''
    
    # Get a mask within time range
    time_mask = (t > start_datetime) & (t < end_datetime)
    dead = (t > start_datetime) & (t < end_datetime) & (sc_data['i'] < 0.4)
    mask = (t > start_datetime) & (t < end_datetime) & (sc_data['i'] >= 0.4)
    
    v_power = 3.5
    i_power = 1.
    
    v_eff = (np.average(sc_data[mask]['v']**v_power))**(1/v_power)
    i_eff = (np.average(sc_data[mask]['i']**i_power))**(1/i_power)
    t_dead = len(t[dead])
    
    if verbose:
        print("v_eff: %f" % v_eff)
        print("v_avg: %f" % np.average(sc_data[mask]['v']))
        print("v_std: %f" % np.std(sc_data[mask]['v']))
        print("")
        print("i_eff: %f" % i_eff)
        print("i_avg: %f" % np.average(sc_data[mask]['i']))
        print("i_std: %f" % np.std(sc_data[mask]['i']))    
        print("")
        print("deadtime: %d" % t_dead)
    
    if plot:
        plt.plot(t[time_mask],sc_data[time_mask]['v'], label='voltage', color='blue')
        if len(t[dead]) > 0:
            plt.scatter(t[dead],sc_data[dead]['v'], label='voltage', color='red')

        plt.axhline(v_eff,ls='--', lw=1.5, color='blue', label = 'v_eff')
        plt.ylabel('Voltage (kV)')
        plt.ylim(0,60)
        
        
        
        plt.twinx()
        plt.plot(t[time_mask],sc_data[time_mask]['i'], label='current', color='green')
        plt.axhline(i_eff,ls='--', lw=1.5, color='green', label = 'i_eff')
        if len(t[dead]) > 0:
            plt.scatter(t[dead],sc_data[dead]['i'], label='deadtime', color='red')
        plt.ylabel('Current (mA)')
        plt.ylim(0,2.5)
        
        plt.legend(loc='lower right')
        plt.gcf().autofmt_xdate()
        plt.show()
        
    return v_eff, i_eff, t_dead

### Gamma calibration functions

In [ ]:
def get_fwhm_pos(d, edge = 'right'):
    '''
    Get index of FWHM, right edge
    '''
    if edge != 'right':
        raise NotImplementedError('Just right edge for now!')
     
    max_val = max(d)
    max_index = d.index(max_val)
    for i in range(max_index, len(d) - max_index):
        if d[i] < max_val*0.5:
            return i
    return -1
    

In [ ]:
def get_compton_energy(e):
    '''
    
    https://en.wikipedia.org/wiki/Compton_edge (but I also checked other places, don't worry)
    '''
    return e*(1-1/(1+(2*e/511.)))

In [ ]:
def poly_1(x, a0, a1):
    return a0 + a1*x

### Other functions

In [ ]:
def draw_box(x, y, **kwargs):
    """Draw rectangle, given x-y boundary tuples"""
    # Arcane syntax of the week: matplotlib's Rectangle...
    plt.gca().add_patch(mpl.patches.Rectangle(
        (x[0], y[0]), x[1] - x[0], y[1] - y[0], facecolor='none', **kwargs))